# 🏈 Sleeper Fantasy League - Master Control Panel

**Complete control over your Sleeper fantasy league data.**

## Quick Start:
1. **First Time**: Run all cells to load your league data
2. **Weekly Updates**: Just update `CURRENT_WEEK` and run all cells
3. **View Data**: Scroll down to see standings, matchups, and analysis

---


## 📦 Setup & Configuration


In [1]:
import sqlite3
import json
import sys
from datetime import datetime
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path().absolute().parent.parent))
sys.path.append(str(Path().absolute().parent / 'scrapers'))

from backend.scrapers.database_league import LeagueDB
from backend.scrapers.scraper_sleeper_league import SleeperLeagueScraper
import os
from dotenv import load_dotenv

# Load environment variables from .env file in root
load_dotenv(Path().absolute().parent.parent / '.env')

# ==================== CONFIGURATION ====================
# Update CURRENT_WEEK each week - that's all you need to change!

SLEEPER_USERNAME = os.getenv("SLEEPER_USERNAME")
LEAGUE_ID = os.getenv("LEAGUE_ID")
SEASON = "2025"
CURRENT_WEEK = 12  # ⬅️ UPDATE THIS WEEKLY!

# Get database paths
NOTEBOOK_DIR = Path().absolute()
BACKEND_DIR = NOTEBOOK_DIR.parent
DB_LEAGUE_PATH = str(BACKEND_DIR / "data" / "databases" / "league.db")

print("✅ Setup Complete!")
print(f"📅 Season {SEASON}, Week {CURRENT_WEEK}")
print(f"👤 User: {SLEEPER_USERNAME}")
print(f"🏈 League ID: {LEAGUE_ID}")
print("\n" + "="*70 + "\n")


✅ Setup Complete!
📅 Season 2025, Week 12
👤 User: sfaizi24
🏈 League ID: 1226433368405585920




## 🔄 Update League Data

**Run this weekly** to get the latest matchups, rosters, and standings.


In [2]:
def update_league_data(week=None):
    """Update league data for the current/specified week."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID in configuration!")
        return
    
    if week is None:
        week = CURRENT_WEEK
    
    print(f"{'='*70}")
    print(f"UPDATING LEAGUE DATA - Week {week}")
    print(f"{'='*70}\n")
    
    with SleeperLeagueScraper(db_path=DB_LEAGUE_PATH) as scraper:
        # Update rosters (W-L records, players)
        print("📊 Updating rosters...")
        rosters = scraper.get_league_rosters(LEAGUE_ID)
        if rosters:
            with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
                db.insert_rosters_batch(rosters, LEAGUE_ID)
                print(f"  ✓ Updated {len(rosters)} rosters\n")
        
        # Update matchups
        print(f"🎮 Updating matchups for week {week}...")
        matchups = scraper.get_league_matchups(LEAGUE_ID, week)
        if matchups:
            with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
                db.insert_matchups_batch(matchups, LEAGUE_ID, week)
                print(f"  ✓ Updated {len(matchups)} matchups\n")
        
        # Update player stats
        print(f"📈 Updating player stats for week {week}...")
        scraper.save_player_stats(SEASON, week, week)
    
    print(f"{'='*70}")
    print(f"✅ UPDATE COMPLETE!")
    print(f"{'='*70}\n")

# Run the update
update_league_data()


UPDATING LEAGUE DATA - Week 12

📊 Updating rosters...
Fetching rosters for league 1226433368405585920...
  ✓ Found 12 rosters/teams
  ✓ Updated 12 rosters

🎮 Updating matchups for week 12...
Fetching matchups for week 12...
  ✓ Found 12 team matchups (6 games)
  ✓ Updated 12 matchups

📈 Updating player stats for week 12...

FETCHING PLAYER STATS

Fetching player stats for 2025 Week 12...
  ✓ Loaded stats for 0 players

PLAYER STATS COMPLETE
Weeks fetched: 0
Total player-week stats: 0

❌ No stats data to save
✅ UPDATE COMPLETE!



## 🏆 League Standings


In [3]:
def show_standings():
    """Display league standings."""
    with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
        rosters = db.get_rosters(LEAGUE_ID)
        
        print(f"{'='*70}")
        print(f"LEAGUE STANDINGS")
        print(f"{'='*70}\n")
        print(f"{'Rank':<6} {'Record':<10} {'Points For':<12} {'Team (Owner)'}") 
        print("-" * 70)
        
        for i, roster in enumerate(rosters, 1):
            team_name = roster.get('team_name') or f"Team {roster['roster_id']}"
            owner = roster.get('display_name', 'Unknown')
            record = f"{roster.get('wins', 0)}-{roster.get('losses', 0)}-{roster.get('ties', 0)}"
            fpts = roster.get('fpts', 0) + roster.get('fpts_decimal', 0)
            
            print(f"{i:<6} {record:<10} {fpts:<12.2f} {team_name} ({owner})")
        
        print(f"\n{'='*70}\n")

show_standings()


LEAGUE STANDINGS

Rank   Record     Points For   Team (Owner)
----------------------------------------------------------------------
1      8-3-0      1414.00      Team 1 (xavierking4)
2      7-4-0      1494.00      Team 8 (sahirsyed30)
3      7-4-0      1380.00      Team 2 (asadrafique)
4      7-4-0      1256.00      Team 12 (sfaizi24)
5      6-5-0      1419.00      Team 3 (amir812)
6      6-5-0      1346.00      Team 5 (TBK41)
7      6-5-0      1344.00      Team 10 (monkeyman966699696)
8      6-5-0      1241.00      Team 11 (Ammady)
9      5-6-0      1248.00      Team 6 (Jibraan)
10     4-7-0      1350.00      Team 7 (mehdidrissi)
11     3-8-0      1128.00      Team 9 (Bilal879)
12     1-10-0     1125.00      Team 4 (umarrahman30)




## 🎮 Current Week Matchups


In [4]:
def show_matchups(week=None):
    """Display matchups for a specific week."""
    if week is None:
        week = CURRENT_WEEK
    
    with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
        matchups = db.get_matchups(LEAGUE_ID, week)
        
        if not matchups:
            print(f"No matchups found for week {week}")
            return
        
        print(f"{'='*70}")
        print(f"MATCHUPS - Week {week}")
        print(f"{'='*70}\n")
        
        # Group by matchup_id
        matchup_groups = {}
        for m in matchups:
            mid = m.get('matchup_id_number')
            if mid not in matchup_groups:
                matchup_groups[mid] = []
            matchup_groups[mid].append(m)
        
        # Display each matchup
        for mid in sorted(matchup_groups.keys()):
            teams = matchup_groups[mid]
            
            if len(teams) == 2:
                team1, team2 = teams
                
                name1 = team1.get('team_name') or team1.get('display_name') or f"Team {team1['roster_id']}"
                name2 = team2.get('team_name') or team2.get('display_name') or f"Team {team2['roster_id']}"
                
                pts1 = team1.get('points', 0)
                pts2 = team2.get('points', 0)
                
                winner = "🏆" if pts1 > pts2 and pts1 > 0 else "  "
                winner2 = "🏆" if pts2 > pts1 and pts2 > 0 else "  "
                
                print(f"Matchup {mid}:")
                print(f"  {winner} {name1:<30} {pts1:>6.2f}")
                print(f"  {winner2} {name2:<30} {pts2:>6.2f}")
                print()
        
        print(f"{'='*70}\n")

show_matchups()


MATCHUPS - Week 12

Matchup 1:
     umarrahman30                     0.00
     TBK41                            0.00

Matchup 2:
     xavierking4                      0.00
     sahirsyed30                      0.00

Matchup 3:
     Jibraan                          0.00
     sfaizi24                         0.00

Matchup 4:
     monkeyman966699696               0.00
     Ammady                           0.00

Matchup 5:
     mehdidrissi                      0.00
     Bilal879                         0.00

Matchup 6:
     asadrafique                      0.00
     amir812                          0.00




---

## 📝 Additional Functions

**Less frequently used - only run when needed**


In [8]:
# ============================================================================
# UTILITY FUNCTIONS - Uncomment to use
# ============================================================================

def show_database_status():
    """Display current database status."""
    with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
        print(f"{'='*70}")
        print(f"DATABASE STATUS")
        print(f"{'='*70}\n")
        
        leagues = db.get_all_leagues()
        print(f"📊 Leagues: {len(leagues)}")
        for league in leagues:
            print(f"   - {league['name']} ({league['season']})")
        
        if LEAGUE_ID:
            users = db.get_users(LEAGUE_ID)
            rosters = db.get_rosters(LEAGUE_ID)
            matchups = db.get_matchups(LEAGUE_ID)
            weeks_with_matchups = len(set(m['week'] for m in matchups)) if matchups else 0
            
            print(f"\n👥 Users: {len(users)}")
            print(f"🏆 Teams: {len(rosters)}")
            print(f"🎮 Matchups: {len(matchups)} across {weeks_with_matchups} weeks")
        
        all_players = db.get_nfl_players()
        print(f"🏈 NFL Players: {len(all_players)}")
        
        all_stats = db.get_player_stats()
        if all_stats:
            weeks_with_stats = len(set(s['week'] for s in all_stats))
            print(f"📈 Player Stats: {len(all_stats)} records across {weeks_with_stats} weeks")
        
        print(f"\n{'='*70}\n")

def show_team_roster(roster_id):
    """Display a team's roster."""
    import ast
    with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
        roster = db.get_roster(roster_id, LEAGUE_ID)
        if not roster:
            print(f"Roster {roster_id} not found")
            return
        
        team_name = roster.get('team_name') or f"Team {roster_id}"
        owner = roster.get('display_name', 'Unknown')
        
        print(f"{'='*70}")
        print(f"{team_name} - {owner}")
        print(f"{'='*70}\n")
        print(f"Record: {roster.get('wins', 0)}-{roster.get('losses', 0)}-{roster.get('ties', 0)}")
        print(f"Points For: {roster.get('fpts', 0):.2f}")
        print(f"Points Against: {roster.get('fpts_against', 0):.2f}")
        
        try:
            players_str = roster.get('players', '[]')
            player_ids = ast.literal_eval(players_str) if isinstance(players_str, str) else players_str
            
            if player_ids:
                print(f"\n📋 Roster ({len(player_ids)} players):")
                print("-" * 70)
                for player_id in player_ids[:20]:
                    player = db.get_nfl_player(player_id)
                    if player:
                        name = player.get('full_name') or 'Unknown'
                        pos = player.get('position') or '??'
                        team = player.get('team') or 'FA'
                        status = player.get('injury_status') or ''
                        status_str = f" ({status})" if status else ""
                        print(f"  {pos:<3} {name:<25} {team:<4}{status_str}")
        except Exception as e:
            print(f"\n⚠ Could not parse roster: {e}")
        
        print(f"\n{'='*70}\n")

def show_bye_weeks(week=None):
    """Display bye weeks."""
    with LeagueDB(db_path=DB_LEAGUE_PATH) as db:
        if week:
            schedules = db.get_schedules(season=SEASON, week=week, is_bye=True)
            teams_on_bye = [s['team'] for s in schedules]
            print(f"{'='*70}")
            print(f"BYE WEEKS - Week {week}")
            print(f"{'='*70}\n")
            if teams_on_bye:
                print(f"Teams on bye: {', '.join(sorted(teams_on_bye))}")
            else:
                print("No teams on bye this week")
        else:
            bye_weeks = db.get_bye_weeks(SEASON)
            print(f"{'='*70}")
            print(f"ALL BYE WEEKS - {SEASON}")
            print(f"{'='*70}\n")
            weeks_dict = {}
            for team, week_num in bye_weeks.items():
                if week_num not in weeks_dict:
                    weeks_dict[week_num] = []
                weeks_dict[week_num].append(team)
            for week_num in sorted(weeks_dict.keys()):
                teams = sorted(weeks_dict[week_num])
                print(f"Week {week_num:2}: {', '.join(teams)}")
        print(f"\n{'='*70}\n")

def initial_data_load():
    """⚠️ ONLY RUN ONCE - Loads all historical data."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID first!")
        return
    
    print(f"{'='*70}")
    print(f"INITIAL DATA LOAD")
    print(f"{'='*70}\n")
    print(f"Loading weeks 1-{CURRENT_WEEK}... This may take 2-3 minutes.\n")
    
    with SleeperLeagueScraper(db_path=DB_LEAGUE_PATH) as scraper:
        scraper.save_league_data(
            league_id=LEAGUE_ID,
            weeks=list(range(1, CURRENT_WEEK + 1)),
            include_transactions=True
        )
        scraper.save_nfl_players()
        scraper.save_player_stats(SEASON, 1, CURRENT_WEEK)
        scraper.save_nfl_schedule(SEASON)
    
    print(f"\n{'='*70}")
    print(f"✅ INITIAL LOAD COMPLETE!")
    print(f"{'='*70}\n")

# Uncomment to use:
# show_database_status()
# show_team_roster(12)  # Change number to view different teams
# show_bye_weeks()  # All bye weeks
# show_bye_weeks(week=CURRENT_WEEK)  # Current week only
# initial_data_load()  # ⚠️ ONLY run once for first-time setup!

print("✅ Utility functions loaded. Uncomment above to use.")


DATABASE STATUS

📊 Leagues: 1
   - Traveling Fantasy Circus 🎪 (2025)

👥 Users: 12
🏆 Teams: 12
🎮 Matchups: 144 across 12 weeks
🏈 NFL Players: 3968
📈 Player Stats: 22315 records across 10 weeks


✅ Utility functions loaded. Uncomment above to use.


---

## 📚 Quick Reference

### Weekly Workflow:
1. Update `CURRENT_WEEK` in cell 2
2. **Kernel → Restart & Run All**
3. View standings and matchups below

### Available Functions:
- `update_league_data()` - Update for current week *(runs automatically)*
- `show_standings()` - View league standings *(runs automatically)*
- `show_matchups(week)` - View matchups *(runs automatically)*
- `show_database_status()` - Check database contents
- `show_team_roster(roster_id)` - View any team's roster
- `show_bye_weeks(week)` - Check bye weeks
- `initial_data_load()` - First time setup only ⚠️

### Tips:
- Games showing 0 points means they haven't been played yet
- Run this notebook every Monday/Tuesday for latest results
- Database path: `backend/data/databases/league.db`

---

**✅ Ready to use! Just hit "Restart & Run All" each week.**
